# Disclaimer
This data has very little context so this notebook is mainly here to mess around with Folium and such.

In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
import folium

In [35]:
from os.path import basename, exists
import os

def download(url):
    file_name = basename(url)
    if exists(file_name): return
    
    from urllib.request import urlretrieve
    local, _ = urlretrieve(url, file_name)
    print('Downloaded ' + local)
    
download('https://fsa-catalogue2.s3.eu-west-2.amazonaws.com/Biotoxin+Results+2021+160322.csv')

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1125)>

In [36]:
biotoxin_results = pd.read_csv('biotoxin-results.csv', parse_dates=['DateSampleCollected'])
biotoxin_results

,SampleNumber,ProductionArea,BedID,LocalAuthority,GridReference,SamplingPoint,DateSampleCollected,SpeciesSampled,PSP_HPLCScreenResult(Detected-ND),PSP_HPLC(Microgram_STXeq_per_kg)SemiquantResult_<400->400,PSP_HPLCResult_(Microgram_STXeq_per_kg)HighValueCalculatedFromMU,TotalOA-DTXs-PTXs(Microgram_OA_eq_per_kg)HighValueCalculatedFromMU,TotalAZAS_(Microgram_AZA1_eq_per_kg)HighValueCalculatedFromMU,TotalYTXS(mg_YTX_eq_per_kg)HighValueCalculatedFromMU,ASP(mgPerkg)
0,BTX/2021/0001,Solent,B24BK,Southampton PHA,SZ57479794,Browndown,2021-04-01,Native oysters,ND,NaN,NaN,<RL,<RL,<RL,<LOQ
1,BTX/2021/0002,Southampton Water,B21AT,Southampton PHA,SU48760530,Hamble Estuary,2021-04-01,Manila clams,ND,NaN,NaN,<RL,<RL,<RL,<LOQ
2,BTX/2021/0004,Morecambe Bay - Roosebeck,B48AX,Barrow-in-Furness BC,SD25996647,Roosebeck North,2021-05-01,Pacific oysters,ND,NaN,NaN,<RL,<RL,<RL,<LOQ
3,BTX/2021/0005,Morecambe Bay - Barrow,B077R,Barrow-in-Furness BC,SD24916365,Foulney,2021-05-01,Mussels,ND,NaN,NaN,<RL,<RL,<RL,<LOQ
4,BTX/2021/0006,Blackwater,B014V,Maldon DC,TL91160795,Goldhanger,2021-05-01,Pacific oysters,ND,NaN,NaN,<RL,<RL,<RL,<LOQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,Insufficient OR Unsuitable sample,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
961,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
962,Note,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
963,1 = Information relating to the origin of samp...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
biotoxin_results.isnull().sum()

SampleNumber                                                            2
ProductionArea                                                         15
BedID                                                                  15
LocalAuthority                                                         15
GridReference                                                          15
SamplingPoint                                                          15
DateSampleCollected                                                    15
SpeciesSampled                                                         15
PSP_HPLCScreenResult(Detected-ND)                                     151
PSP_HPLC(Microgram_STXeq_per_kg)SemiquantResult_<400->400             870
PSP_HPLCResult_(Microgram_STXeq_per_kg)HighValueCalculatedFromMU      962
TotalOA-DTXs-PTXs(Microgram_OA_eq_per_kg)HighValueCalculatedFromMU     63
TotalAZAS_(Microgram_AZA1_eq_per_kg)HighValueCalculatedFromMU          63
TotalYTXS(mg_YTX_eq_per_kg)HighValueCa

I will filter out some of these columns as they have too many null values or are totally unknown.

In [38]:
biotoxin_results = biotoxin_results[['SampleNumber', 'ProductionArea', 'BedID', 'LocalAuthority',
                                     'GridReference', 'SamplingPoint', 'DateSampleCollected', 'SpeciesSampled']]
biotoxin_results

,SampleNumber,ProductionArea,BedID,LocalAuthority,GridReference,SamplingPoint,DateSampleCollected,SpeciesSampled
0,BTX/2021/0001,Solent,B24BK,Southampton PHA,SZ57479794,Browndown,2021-04-01,Native oysters
1,BTX/2021/0002,Southampton Water,B21AT,Southampton PHA,SU48760530,Hamble Estuary,2021-04-01,Manila clams
2,BTX/2021/0004,Morecambe Bay - Roosebeck,B48AX,Barrow-in-Furness BC,SD25996647,Roosebeck North,2021-05-01,Pacific oysters
3,BTX/2021/0005,Morecambe Bay - Barrow,B077R,Barrow-in-Furness BC,SD24916365,Foulney,2021-05-01,Mussels
4,BTX/2021/0006,Blackwater,B014V,Maldon DC,TL91160795,Goldhanger,2021-05-01,Pacific oysters
...,...,...,...,...,...,...,...,...
960,Insufficient OR Unsuitable sample,NaN,NaN,NaN,NaN,NaN,NaT,NaN
961,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
962,Note,NaN,NaN,NaN,NaN,NaN,NaT,NaN
963,1 = Information relating to the origin of samp...,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [39]:
biotoxin_results = biotoxin_results.head(len(biotoxin_results) - 15)
biotoxin_results

,SampleNumber,ProductionArea,BedID,LocalAuthority,GridReference,SamplingPoint,DateSampleCollected,SpeciesSampled
0,BTX/2021/0001,Solent,B24BK,Southampton PHA,SZ57479794,Browndown,2021-04-01,Native oysters
1,BTX/2021/0002,Southampton Water,B21AT,Southampton PHA,SU48760530,Hamble Estuary,2021-04-01,Manila clams
2,BTX/2021/0004,Morecambe Bay - Roosebeck,B48AX,Barrow-in-Furness BC,SD25996647,Roosebeck North,2021-05-01,Pacific oysters
3,BTX/2021/0005,Morecambe Bay - Barrow,B077R,Barrow-in-Furness BC,SD24916365,Foulney,2021-05-01,Mussels
4,BTX/2021/0006,Blackwater,B014V,Maldon DC,TL91160795,Goldhanger,2021-05-01,Pacific oysters
...,...,...,...,...,...,...,...,...
945,BTX/2021/3342,Poole,B54CI,Bournemouth Poole Christchurch Council,SZ01818871,West Brownsea 1,2021-12-14,Mussels
946,BTX/2021/3348,Brixham,B082B,Torbay BC,SX90965741,Fishcombe SW corner,2021-12-14,Mussels
947,BTX/2021/3349,Lyme Bay,B090M,Torbay BC,SY13687543,Site 1,2021-12-14,Mussels
948,BTX/2021/3350,Exe,B26BH,Teignbridge DC,SX97948072,Cockwood Harbour,2021-12-14,Mussels


Removed some context information at the bottom that wasn't needed.

In [40]:
biotoxin_results['SpeciesSampled'].value_counts()

Mussels            495
Pacific oysters    221
Cockles             92
Queen scallops      38
Native oysters      32
Hard clams          24
Surf clams          16
Manila clams        14
Pacific Oysters     12
King scallops        6
Name: SpeciesSampled, dtype: int64

In [41]:
biotoxin_results = biotoxin_results[biotoxin_results['GridReference'] != 'Unverified']
biotoxin_results

,SampleNumber,ProductionArea,BedID,LocalAuthority,GridReference,SamplingPoint,DateSampleCollected,SpeciesSampled
0,BTX/2021/0001,Solent,B24BK,Southampton PHA,SZ57479794,Browndown,2021-04-01,Native oysters
1,BTX/2021/0002,Southampton Water,B21AT,Southampton PHA,SU48760530,Hamble Estuary,2021-04-01,Manila clams
2,BTX/2021/0004,Morecambe Bay - Roosebeck,B48AX,Barrow-in-Furness BC,SD25996647,Roosebeck North,2021-05-01,Pacific oysters
3,BTX/2021/0005,Morecambe Bay - Barrow,B077R,Barrow-in-Furness BC,SD24916365,Foulney,2021-05-01,Mussels
4,BTX/2021/0006,Blackwater,B014V,Maldon DC,TL91160795,Goldhanger,2021-05-01,Pacific oysters
...,...,...,...,...,...,...,...,...
945,BTX/2021/3342,Poole,B54CI,Bournemouth Poole Christchurch Council,SZ01818871,West Brownsea 1,2021-12-14,Mussels
946,BTX/2021/3348,Brixham,B082B,Torbay BC,SX90965741,Fishcombe SW corner,2021-12-14,Mussels
947,BTX/2021/3349,Lyme Bay,B090M,Torbay BC,SY13687543,Site 1,2021-12-14,Mussels
948,BTX/2021/3350,Exe,B26BH,Teignbridge DC,SX97948072,Cockwood Harbour,2021-12-14,Mussels


In [42]:
from OSGridConverter import grid2latlong, OSGridReference
from OSGridConverter.base import OSGridError

random_grid_ref = OSGridReference('NN8544249023')

def grid_to_latlong(grid_ref):
    try:
        grid = grid2latlong(grid_ref)
        return grid
    except OSGridError:
        print(grid_ref, ' is invalid.')
        return random_grid_ref.toLatLong(tag='WGS84')
    
biotoxin_results['Latitude'] = biotoxin_results['GridReference'].apply(lambda grid_ref: grid_to_latlong(grid_ref).latitude)
biotoxin_results['Longitude'] = biotoxin_results['GridReference'].apply(lambda grid_ref: grid_to_latlong(grid_ref).longitude)
biotoxin_results

SH763087922  is invalid.
Unverified (SY13687543)  is invalid.
SJ1768056  is invalid.
SN365407481  is invalid.
TR10374668224  is invalid.
SN360877726  is invalid.
SZ56490988710  is invalid.
SN459730001  is invalid.
SH763087922  is invalid.
Unverified (SY13687543)  is invalid.
SJ1768056  is invalid.
SN365407481  is invalid.
TR10374668224  is invalid.
SN360877726  is invalid.
SZ56490988710  is invalid.
SN459730001  is invalid.


<ipython-input-42-724a2e2c3039>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biotoxin_results['Latitude'] = biotoxin_results['GridReference'].apply(lambda grid_ref: grid_to_latlong(grid_ref).latitude)
<ipython-input-42-724a2e2c3039>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biotoxin_results['Longitude'] = biotoxin_results['GridReference'].apply(lambda grid_ref: grid_to_latlong(grid_ref).longitude)


,SampleNumber,ProductionArea,BedID,LocalAuthority,GridReference,SamplingPoint,DateSampleCollected,SpeciesSampled,Latitude,Longitude
0,BTX/2021/0001,Solent,B24BK,Southampton PHA,SZ57479794,Browndown,2021-04-01,Native oysters,50.778202,-1.186208
1,BTX/2021/0002,Southampton Water,B21AT,Southampton PHA,SU48760530,Hamble Estuary,2021-04-01,Manila clams,50.845179,-1.308775
2,BTX/2021/0004,Morecambe Bay - Roosebeck,B48AX,Barrow-in-Furness BC,SD25996647,Roosebeck North,2021-05-01,Pacific oysters,54.088798,-3.133078
3,BTX/2021/0005,Morecambe Bay - Barrow,B077R,Barrow-in-Furness BC,SD24916365,Foulney,2021-05-01,Mussels,54.063302,-3.148891
4,BTX/2021/0006,Blackwater,B014V,Maldon DC,TL91160795,Goldhanger,2021-05-01,Pacific oysters,51.737652,0.768798
...,...,...,...,...,...,...,...,...,...,...
945,BTX/2021/3342,Poole,B54CI,Bournemouth Poole Christchurch Council,SZ01818871,West Brownsea 1,2021-12-14,Mussels,50.698034,-1.975736
946,BTX/2021/3348,Brixham,B082B,Torbay BC,SX90965741,Fishcombe SW corner,2021-12-14,Mussels,50.406418,-3.536071
947,BTX/2021/3349,Lyme Bay,B090M,Torbay BC,SY13687543,Site 1,2021-12-14,Mussels,50.572217,-3.220467
948,BTX/2021/3350,Exe,B26BH,Teignbridge DC,SX97948072,Cockwood Harbour,2021-12-14,Mussels,50.617233,-3.444215


In [43]:
random_lat_long = random_grid_ref.toLatLong(tag='WGS84')
biotoxin_results = biotoxin_results[biotoxin_results['Latitude'] != random_lat_long.latitude]
biotoxin_results = biotoxin_results[biotoxin_results['Longitude'] != random_lat_long.longitude]

In [44]:
biotoxin_results

,SampleNumber,ProductionArea,BedID,LocalAuthority,GridReference,SamplingPoint,DateSampleCollected,SpeciesSampled,Latitude,Longitude
0,BTX/2021/0001,Solent,B24BK,Southampton PHA,SZ57479794,Browndown,2021-04-01,Native oysters,50.778202,-1.186208
1,BTX/2021/0002,Southampton Water,B21AT,Southampton PHA,SU48760530,Hamble Estuary,2021-04-01,Manila clams,50.845179,-1.308775
2,BTX/2021/0004,Morecambe Bay - Roosebeck,B48AX,Barrow-in-Furness BC,SD25996647,Roosebeck North,2021-05-01,Pacific oysters,54.088798,-3.133078
3,BTX/2021/0005,Morecambe Bay - Barrow,B077R,Barrow-in-Furness BC,SD24916365,Foulney,2021-05-01,Mussels,54.063302,-3.148891
4,BTX/2021/0006,Blackwater,B014V,Maldon DC,TL91160795,Goldhanger,2021-05-01,Pacific oysters,51.737652,0.768798
...,...,...,...,...,...,...,...,...,...,...
945,BTX/2021/3342,Poole,B54CI,Bournemouth Poole Christchurch Council,SZ01818871,West Brownsea 1,2021-12-14,Mussels,50.698034,-1.975736
946,BTX/2021/3348,Brixham,B082B,Torbay BC,SX90965741,Fishcombe SW corner,2021-12-14,Mussels,50.406418,-3.536071
947,BTX/2021/3349,Lyme Bay,B090M,Torbay BC,SY13687543,Site 1,2021-12-14,Mussels,50.572217,-3.220467
948,BTX/2021/3350,Exe,B26BH,Teignbridge DC,SX97948072,Cockwood Harbour,2021-12-14,Mussels,50.617233,-3.444215


In [45]:
biotoxin_results = biotoxin_results.drop(['GridReference', 'LocalAuthority', 'BedID', 'SampleNumber'], axis=1)
biotoxin_results.to_csv('cleaned_biotoxin_results.csv')

In [46]:
from folium import Marker

biotoxin_map = folium.Map(location=[54, -1], tiles='openstreetmap', zoom_start=6)

for _, row in biotoxin_results.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['SpeciesSampled']).add_to(biotoxin_map)
    
biotoxin_map

In [47]:
biotoxin_results['SpeciesSampled'].value_counts()

Mussels            474
Pacific oysters    220
Cockles             88
Queen scallops      38
Native oysters      31
Hard clams          24
Surf clams          16
Manila clams        14
Pacific Oysters     12
King scallops        6
Name: SpeciesSampled, dtype: int64